In [ ]:
input_path = "../data/blooms.csv"
checkpoint = "lightning_logs/version_9/checkpoints/epoch=49-step=1200.ckpt"
output_path = "../data/embeddings.csv"

In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from transformer import LinearData, Transformer, LitTransformer

# use the data from ../generate
samples_df = pd.read_csv(input_path)
dataset = LinearData(samples_df)
loader = DataLoader(dataset, batch_size=64)
lit_model = LitTransformer.load_from_checkpoint(checkpoint, model = Transformer())

z = []
i = 1

with torch.no_grad():
  for x, y in loader:
    print(i)
    i += 1
    z_, _ = lit_model.model(x)
    z.append(z_[0])

In [ ]:
import numpy as np

z0 = torch.cat(z, dim=0)
z0 = z0.view(z0.size(0), -1).numpy()
np.savetxt(output_path, z0, delimiter=',')